# Query and Process Financial Data

This notebook queries and loads the financial data, preprocesses it and calculates valuable features for the machine learning models in the notebook 'analysis.ipynb'. 

Some parts of the code in this notebook were generated with the assistance of ChatGPT, an AI language model developed by OpenAI. However, every single line of code was manually reviewed by the author of this notebook.

### Table of Content

* [1. Query the data](#chapter1)
* [2. Read the data from csv files](#chapter2)
* [3. Feature Engineering](#chapter3)
* [4. Export the financial features](#chapter4)

## 1. Query the data <a class='anchor' id='chapter1'></a>

This section queries historical prices, balance sheets, income statements, and cashflow statements.

### 1.1 Historical stock prices from Yahoo Finance

Yahoo Finance was used to query the historical prices for the following list of stocks. However, not all of these stocks are needed for further analysis in this project. Due to the limitations of the Reddit API, it was only possible to query sentiment data for the second (commented) list of symbols.

<table>
  <tr>
    <td>
      <ul>
        <li>Apple Inc. (AAPL)</li>
        <li>Alphabet Inc. (GOOG)</li>
        <li>Amazon.com Inc. (AMZN)</li>
        <li>Advanced Micro Devices Inc. (AMD)</li>
        <li>Microsoft Corporation (MSFT)</li>
        <li>Tesla Inc. (TSLA)</li>
        <li>Meta Platforms Inc. (META)</li>
        <li>Netflix Inc. (NFLX)</li>
        <li>Adobe Inc. (ADBE)</li>
        <li>Cisco Systems Inc. (CSCO)</li>
        <li>Qualcomm Inc. (QCOM)</li>
        <li>Intel Corporation (INTC)</li>
        <li>CrowdStrike Holdings Inc. (CRWD)</li>
        <li>Salesforce.com Inc. (CRM)</li>
        <li>Alibaba Group Holding Ltd. (BABA)</li>
        <li>PayPal Holdings Inc. (PYPL)</li>
        <li>The Trade Desk Inc. (TTD)</li>
        <li>Electronic Arts Inc. (EA)</li>
        <li>NVIDIA Corporation (NVDA)</li>
        <li>Zillow Group Inc. (ZG)</li>
        <li>Match Group Inc. (MTCH)</li>
        <li>Cloudflare Inc. (NET)</li>
        <li>Gen Digital Inc. (GEN)</li>
      </ul>
    </td>
    <td>
      <ul>
        <li>Yelp Inc. (YELP)</li>
        <li>Broadcom Inc. (AVGO)</li>
        <li>ASML Holding NV (ASML)</li>
        <li>Oracle Corporation (ORCL)</li>
        <li>SAP SE (SAP)</li>
        <li>International Business Machines Corporation (IBM)</li>
        <li>Intuit Inc. (INTU)</li>
        <li>Uber Technologies Inc. (UBER)</li>
        <li>ON Semiconductor Corporation (ON)</li>
        <li>Dell Technologies Inc. (DELL)</li>
        <li>Snap Inc. (SNAP)</li>
        <li>MicroStrategy Inc. (MSTR)</li>
        <li>Entegris Inc. (ENTG)</li>
        <li>NXP Semiconductors NV (NXPI)</li>
        <li>Seagate Technology Holdings PLC (STX)</li>
        <li>Micron Technology Inc. (MU)</li>
        <li>Qorvo Inc. (QRVO)</li>
        <li>Block Inc. (SQ)</li>
        <li>Monolithic Power Systems Inc. (MPWR)</li>
        <li>Analog Devices Inc. (ADI)</li>
        <li>Atlassian Corporation PLC (TEAM)</li>
        <li>Okta Inc. (OKTA)</li>
        <li>Akamai Technologies Inc. (AKAM)</li>
      </ul>
    </td>
    <td>
      <ul>
        <li>Western Digital Corporation (WDC)</li>
        <li>Applied Materials Inc. (AMAT)</li>
        <li>Pinduoduo Inc. (PDD)</li>
        <li>ServiceNow Inc. (NOW)</li>
        <li>Texas Instruments Inc. (TXN)</li>
        <li>Arm Holdings PLC (ARM)</li>
        <li>Lam Research Corporation (LRCX)</li>
        <li>Booking Holdings Inc. (BKNG)</li>
        <li>Sony Group Corporation (SONY)</li>
        <li>Automatic Data Processing Inc. (ADP)</li>
        <li>KLA Corporation (KLAC)</li>
        <li>Shopify Inc. (SHOP)</li>
        <li>Palo Alto Networks Inc. (PANW)</li>
        <li>Fiverr International Ltd. (FI)</li>
        <li>Synopsys Inc. (SNPS)</li>
        <li>Cadence Design Systems Inc. (CDNS)</li>
        <li>MercadoLibre Inc. (MELI)</li>
        <li>Equinix Inc. (EQIX)</li>
        <li>Workday Inc. (WDAY)</li>
        <li>NetEase Inc. (NTES)</li>
        <li>Marvell Technology Inc. (MRVL)</li>
        <li>Roper Technologies Inc. (ROP)</li>
        <li>DoorDash Inc. (DASH)</li>
      </ul>
    </td>
  </tr>
</table>

In [ ]:
import yfinance as yf
import pandas as pd

# Define stocks for which the data should be queried
symbols = ["AAPL", "GOOG", "AMZN", "AMD", "MSFT", "TSLA", "META", "NFLX", 
        "ADBE", "CSCO", "QCOM", "INTC", "CRWD", "CRM", "BABA", "PYPL", 
        "TTD", "EA", "NVDA", "ZG", "MTCH", "YELP", "AVGO",
        "ASML", "ORCL", "SAP", "IBM", "INTU", "UBER",
        "ON", "DELL", "SNAP", "MSTR", "ENTG",
        "NXPI", "STX", "MU", "QRVO", "SQ", "MPWR", "ALAB", "TEAM",
        "NET", "GEN", "OKTA", "AKAM", "WDC", "AMAT", "PDD", "NOW", "TXN", "ARM",
        "LRCX", "BKNG", "SONY", "ADP", "ADI", "KLAC", "SHOP", "PANW",
        "FI", "SNPS", "CDNS", "MELI", "EQIX", "WDAY", "NTES", "MRVL", "ROP", "DASH"]

# The following list of stocks would have been sufficient for analysis. Due to the limitations of the Reddit API, it was only possible to query Reddit data for all stocks in the list above.
#symbols = ["AAPL", "GOOG", "AMZN", "AMd", "MSFT", "CSCO", "CRWD", "UBER", "IBM", "ORCL", "TSLA", "META", "NFLX", "ADBE", "NVDA", "PYPL", "QCOM", "INTC", "CRM", "TTD", "EA"]

# Create an empty list for the price dataframes
all_prices = []

# Make the query from yfinance for each symbol
for symbol in symbols:
        # Downloading historical stock prices
        prices = yf.download(symbol,
                                start="2010-01-01",
                                end="2024-04-01",
                                progress=False)
        prices['Symbol'] = symbol  # Add a column for the symbol
        all_prices.append(prices)
combined_prices = pd.concat(all_prices)
combined_prices.to_csv('./data/hist_prices_2010.csv')

### 1.1 Income statements, balance sheets and cashflow statements from Alpha Vantage

This section queries quartlerly financial reports from Alpha Vantage. Due to the limitation of 25 queries per day, a function was implemented to query and save data for a maximum of 8 stocks per day.

In [ ]:
from alpha_vantage.fundamentaldata import FundamentalData

# Define the API key
api_key = 'PLEASE_INSERT_YOUR_API_KEY'
fd = FundamentalData(api_key)

def download_fin_reports(symbols):
    # Create empty lists for the dataframes
    all_income_statements = []
    all_balance_sheets = []
    all_cashflow_statements = []
    
    for symbol in symbols:
        
        # Fetch the quarterly income statements
        income_statements, _ = fd.get_income_statement_quarterly(symbol=symbol)
        income_statements['Symbol'] = symbol
        all_income_statements.append(income_statements)

        # Fetch the quarterly balance sheets
        balance_sheets, _ = fd.get_balance_sheet_quarterly(symbol=symbol)
        balance_sheets['Symbol'] = symbol
        all_balance_sheets.append(balance_sheets)

        # Fetch the quarterly cashflow statements
        cashflow_statements, _ = fd.get_cash_flow_quarterly(symbol=symbol)
        cashflow_statements['Symbol'] = symbol
        all_cashflow_statements.append(cashflow_statements)

    # Concatenate all income statements, balance sheets and cashflow statements into three dataframes
    combined_is = pd.concat(all_income_statements)
    combined_bs = pd.concat(all_balance_sheets)
    combined_cs = pd.concat(all_cashflow_statements)
    
    # Save the dataframes as csv-file (different number from 1 to 15 for each query)
    combined_is.to_csv('./data/income_statements/income_statements_1.csv', index=False)
    combined_bs.to_csv('./data/balance_sheets/balance_sheets_1.csv', index=False)
    combined_cs.to_csv('./data/cashflow_statements/cashflow_statements_1.csv', index=False)

# List of stock symbols
symbols = ["AAPL", "GOOG", "AMZN", "AMD", "MSFT", "TSLA", "META", "NFLX"]   # symbols of the first query, these must be replaced to query data for other stocks

# Stocks for which the report data was queried
# 1: "AAPL", "GOOG", "AMZN", "AMD", "MSFT", "TSLA", "META", "NFLX"
# 2: "ADBE", "CSCO", "QCOM", "INTC", "CRWD", "CRM", "BABA", "PYPL"
# 3: "TTD", "EA", "NVDA"
# 4: "ZG", "MTCH", "YELP", "AVGO"
# 5: "ASML", "ORCL"
# 6: "SAP", "IBM", "INTU", "UBER"
# 7: "ON", "DELL", "SNAP", "MSTR", "ENTG"
# 8: "NXPI", "STX", "MU"
# 9: "QRVO", "SQ", "MPWR", "ALAB", "TEAM"
# 10: "NET", "GEN", "OKTA", "AKAM", "WDC"
# 11: "AMAT", "PDD", "NOW", "TXN", "ARM"
# 12: "LRCX", "BKNG", "SONY"
# 13: "ADP", "ADI", "KLAC", "SHOP", "PANW"
# 14: "FI", "SNPS", "CDNS", "MELI", "EQIX"
# 15: "WDAY", "NTES", "MRVL", "ROP", "DASH"

# Downloading data
download_fin_reports(symbols)

## 2. Read the data from csv files <a class='anchor' id='chapter2'></a>

The previously queried financial data and the manually downloaded FED fund rates are converted to data frames for further processing.

In [3]:
import pandas as pd

# Historical prices
hist_prices = pd.read_csv('./data/hist_prices_2010.csv')

# Income statements, balance sheets and cashflow statements
income_statements_1 = pd.read_csv('./data/income_statements/income_statements_1.csv')
balance_sheets_1 = pd.read_csv('./data/balance_sheets/balance_sheets_1.csv')
cashflow_statements_1 = pd.read_csv('./data/cashflow_statements/cashflow_statements_1.csv')
income_statements_2 = pd.read_csv('./data/income_statements/income_statements_2.csv')
balance_sheets_2 = pd.read_csv('./data/balance_sheets/balance_sheets_2.csv')
cashflow_statements_2 = pd.read_csv('./data/cashflow_statements/cashflow_statements_2.csv')
income_statements_3 = pd.read_csv('./data/income_statements/income_statements_3.csv')
balance_sheets_3 = pd.read_csv('./data/balance_sheets/balance_sheets_3.csv')
cashflow_statements_3 = pd.read_csv('./data/cashflow_statements/cashflow_statements_3.csv')
income_statements_4 = pd.read_csv('./data/income_statements/income_statements_4.csv')
balance_sheets_4 = pd.read_csv('./data/balance_sheets/balance_sheets_4.csv')
cashflow_statements_4 = pd.read_csv('./data/cashflow_statements/cashflow_statements_4.csv')
income_statements_5 = pd.read_csv('./data/income_statements/income_statements_5.csv')
balance_sheets_5 = pd.read_csv('./data/balance_sheets/balance_sheets_5.csv')
cashflow_statements_5 = pd.read_csv('./data/cashflow_statements/cashflow_statements_5.csv')
income_statements_6 = pd.read_csv('./data/income_statements/income_statements_6.csv')
balance_sheets_6 = pd.read_csv('./data/balance_sheets/balance_sheets_6.csv')
cashflow_statements_6 = pd.read_csv('./data/cashflow_statements/cashflow_statements_6.csv')
income_statements_7 = pd.read_csv('./data/income_statements/income_statements_7.csv')
balance_sheets_7 = pd.read_csv('./data/balance_sheets/balance_sheets_7.csv')
cashflow_statements_7 = pd.read_csv('./data/cashflow_statements/cashflow_statements_7.csv')
income_statements_8 = pd.read_csv('./data/income_statements/income_statements_8.csv')
balance_sheets_8 = pd.read_csv('./data/balance_sheets/balance_sheets_8.csv')
cashflow_statements_8 = pd.read_csv('./data/cashflow_statements/cashflow_statements_8.csv')
income_statements_9 = pd.read_csv('./data/income_statements/income_statements_9.csv')
balance_sheets_9 = pd.read_csv('./data/balance_sheets/balance_sheets_9.csv')
cashflow_statements_9 = pd.read_csv('./data/cashflow_statements/cashflow_statements_9.csv')
income_statements_10 = pd.read_csv('./data/income_statements/income_statements_10.csv')
balance_sheets_10 = pd.read_csv('./data/balance_sheets/balance_sheets_10.csv')
cashflow_statements_10 = pd.read_csv('./data/cashflow_statements/cashflow_statements_10.csv')
income_statements_11 = pd.read_csv('./data/income_statements/income_statements_11.csv')
balance_sheets_11 = pd.read_csv('./data/balance_sheets/balance_sheets_11.csv')
cashflow_statements_11 = pd.read_csv('./data/cashflow_statements/cashflow_statements_11.csv')
income_statements_12 = pd.read_csv('./data/income_statements/income_statements_12.csv')
balance_sheets_12 = pd.read_csv('./data/balance_sheets/balance_sheets_12.csv')
cashflow_statements_12 = pd.read_csv('./data/cashflow_statements/cashflow_statements_12.csv')
income_statements_13 = pd.read_csv('./data/income_statements/income_statements_13.csv')
balance_sheets_13 = pd.read_csv('./data/balance_sheets/balance_sheets_13.csv')
cashflow_statements_13 = pd.read_csv('./data/cashflow_statements/cashflow_statements_13.csv')
income_statements_14 = pd.read_csv('./data/income_statements/income_statements_14.csv')
balance_sheets_14 = pd.read_csv('./data/balance_sheets/balance_sheets_14.csv')
cashflow_statements_14 = pd.read_csv('./data/cashflow_statements/cashflow_statements_14.csv')
income_statements_15 = pd.read_csv('./data/income_statements/income_statements_15.csv')
balance_sheets_15 = pd.read_csv('./data/balance_sheets/balance_sheets_15.csv')
cashflow_statements_15 = pd.read_csv('./data/cashflow_statements/cashflow_statements_15.csv')

# Put the dataframes together
income_statements = pd.concat([income_statements_1, income_statements_2, income_statements_3, income_statements_4, income_statements_5, 
                        income_statements_6, income_statements_7, income_statements_8, income_statements_9, income_statements_10, income_statements_11,
                        income_statements_12, income_statements_13, income_statements_14, income_statements_15])

balance_sheets = pd.concat([balance_sheets_1, balance_sheets_2, balance_sheets_3, balance_sheets_4, balance_sheets_5, 
                        balance_sheets_6, balance_sheets_7, balance_sheets_8, balance_sheets_9, balance_sheets_10,
                        balance_sheets_11, balance_sheets_12, balance_sheets_13, balance_sheets_14, balance_sheets_15])

cashflow_statements = pd.concat([cashflow_statements_1, cashflow_statements_2, cashflow_statements_3, cashflow_statements_4, cashflow_statements_5, 
                        cashflow_statements_6, cashflow_statements_7, cashflow_statements_8, cashflow_statements_9, cashflow_statements_10,
                        cashflow_statements_11, cashflow_statements_12, cashflow_statements_13, cashflow_statements_14, cashflow_statements_15])

# Reset the indices
hist_prices = hist_prices.reset_index(drop=True)
income_statements = income_statements.reset_index(drop=True)
balance_sheets = balance_sheets.reset_index(drop=True)
cashflow_statements = cashflow_statements.reset_index(drop=True)

# Read the Federal Funds Effective Rate (csv-file from https://fred.stlouisfed.org/series/FEDFUNDS)
fedfunds = pd.read_csv('./data/FEDFUNDS.csv')

In [4]:
# Have a look at the historical price data
hist_prices

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,2010-01-04,7.622500,7.660714,7.585000,7.643214,6.470742,493729600,AAPL
1,2010-01-05,7.664286,7.699643,7.616071,7.656429,6.481927,601904800,AAPL
2,2010-01-06,7.656429,7.686786,7.526786,7.534643,6.378824,552160000,AAPL
3,2010-01-07,7.562500,7.571429,7.466071,7.520714,6.367032,477131200,AAPL
4,2010-01-08,7.510714,7.571429,7.466429,7.570714,6.409362,447610800,AAPL
...,...,...,...,...,...,...,...,...
224766,2024-03-22,136.809998,137.750000,135.710007,137.240005,137.240005,1387100,DASH
224767,2024-03-25,137.050003,138.899994,136.740005,137.820007,137.820007,2162800,DASH
224768,2024-03-26,138.440002,140.929993,138.089996,140.259995,140.259995,3506600,DASH
224769,2024-03-27,142.550003,143.339996,138.050003,138.880005,138.880005,2171900,DASH


In [5]:
# Have a look at the income statement data
income_statements

,fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,...,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,Symbol
0,2023-12-31,USD,5.485500e+10,1.195750e+11,7.150600e+10,6.472000e+10,4.037300e+10,6.786000e+09,7.696000e+09,1.448200e+10,...,2.848000e+09,4.032300e+10,6.407000e+09,NaN,3.391600e+10,3.599000e+10,4.037300e+10,4.322100e+10,3.391600e+10,AAPL
1,2023-09-30,USD,4.042700e+10,8.849600e+10,5.522200e+10,4.907100e+10,2.696900e+10,6.151000e+09,7.307000e+09,1.345800e+10,...,2.653000e+09,2.699800e+10,4.042000e+09,1.002000e+09,2.295600e+10,2.330500e+10,2.800000e+10,2.962200e+10,2.295600e+10,AAPL
2,2023-06-30,USD,3.641300e+10,8.079900e+10,5.135700e+10,4.538400e+10,2.299800e+10,5.973000e+09,7.442000e+09,1.341500e+10,...,3.052000e+09,2.273300e+10,2.852000e+09,9.980000e+08,1.988100e+10,1.982600e+10,2.373100e+10,2.605000e+10,1.988100e+10,AAPL
3,2023-03-31,USD,4.197600e+10,9.483600e+10,5.286000e+10,5.286000e+10,2.831800e+10,6.201000e+09,7.457000e+09,1.365800e+10,...,2.898000e+09,2.838200e+10,4.222000e+09,NaN,2.416000e+10,2.532600e+10,2.931200e+10,3.121600e+10,2.416000e+10,AAPL
4,2022-12-31,USD,5.033200e+10,1.161510e+11,7.342900e+10,6.682200e+10,3.601600e+10,6.607000e+09,7.709000e+09,1.431600e+10,...,2.916000e+09,3.562300e+10,5.625000e+09,1.003000e+09,2.999800e+10,2.819500e+10,3.662600e+10,3.893200e+10,2.999800e+10,AAPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3585,2021-03-31,USD,-3.600000e+07,1.049000e+09,1.085000e+09,5.630000e+08,-9.900000e+07,5.020000e+08,8.200000e+07,5.880000e+08,...,4.000000e+06,-1.090000e+08,1.000000e+06,1.200000e+07,-1.100000e+08,-1.100000e+08,-9.700000e+07,-9.300000e+07,-1.100000e+08,DASH
3586,2020-12-31,USD,5.010000e+08,9.700000e+08,4.690000e+08,4.690000e+08,-3.050000e+08,2.190000e+08,2.090000e+08,8.060000e+08,...,3.100000e+07,-3.110000e+08,1.000000e+06,NaN,-3.120000e+08,-3.130000e+08,-3.010000e+08,-2.700000e+08,-3.120000e+08,DASH
3587,2020-09-30,USD,4.970000e+08,8.790000e+08,3.820000e+08,3.820000e+08,-3.500000e+07,1.670000e+08,4.100000e+07,5.320000e+08,...,3.400000e+07,-4.200000e+07,1.000000e+06,NaN,-4.300000e+07,-4.400000e+07,-3.300000e+07,1.000000e+06,-4.300000e+07,DASH
3588,2020-06-30,USD,3.520000e+08,6.750000e+08,3.230000e+08,3.230000e+08,2.700000e+07,8.800000e+07,3.800000e+07,3.250000e+08,...,3.100000e+07,2.300000e+07,0.000000e+00,NaN,2.300000e+07,2.200000e+07,3.200000e+07,6.300000e+07,2.300000e+07,DASH


In [6]:
# Have a look at the balance sheet data
balance_sheets

,fiscalDateEnding,reportedCurrency,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,...,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding,Symbol
0,2023-12-31,USD,3.535140e+11,1.436920e+11,4.076000e+10,7.310000e+10,6.511000e+09,5.010200e+10,2.098220e+11,4.366600e+10,...,9.508800e+10,1.079980e+11,5.461100e+10,5.035300e+10,7.410000e+10,NaN,8.242000e+09,7.523600e+10,1.546022e+10,AAPL
1,2023-09-30,USD,3.525830e+11,1.435660e+11,2.996500e+10,6.155500e+10,6.331000e+09,6.098500e+10,2.090170e+11,4.371500e+10,...,9.528100e+10,1.110880e+11,5.882900e+10,4.984800e+10,6.214600e+10,NaN,-2.140000e+08,7.381200e+10,1.555006e+10,AAPL
2,2023-06-30,USD,3.350380e+11,1.226590e+11,2.840800e+10,6.248200e+10,7.351000e+09,3.918600e+10,2.123790e+11,4.355000e+10,...,9.807100e+10,1.092930e+11,5.889700e+10,5.173000e+10,6.027400e+10,NaN,1.408000e+09,7.066700e+10,1.564787e+10,AAPL
3,2023-03-31,USD,3.321600e+11,1.129130e+11,2.468700e+10,2.468700e+10,7.482000e+09,NaN,2.192470e+11,4.339800e+10,...,9.704100e+10,1.096150e+11,5.642500e+10,5.288600e+10,6.215800e+10,NaN,4.336000e+09,6.956800e+10,1.572341e+10,AAPL
4,2022-12-31,USD,3.467470e+11,1.287770e+11,2.053500e+10,5.135500e+10,6.820000e+09,5.418000e+10,2.179700e+11,4.295100e+10,...,9.962700e+10,1.111430e+11,5.989300e+10,5.310700e+10,5.672700e+10,NaN,3.240000e+09,6.639900e+10,1.584241e+10,AAPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3417,2021-12-31,USD,6.809000e+09,4.565000e+09,2.504000e+09,3.757000e+09,NaN,3.490000e+08,1.470000e+09,4.020000e+08,...,NaN,3.990000e+08,1.390000e+09,9.000000e+06,4.667000e+09,NaN,-2.081000e+09,0.000000e+00,3.445570e+08,DASH
3418,2021-09-30,USD,6.358000e+09,4.707000e+09,2.861000e+09,4.160000e+09,NaN,2.860000e+08,1.328000e+09,3.550000e+08,...,NaN,3.660000e+08,1.067000e+09,1.400000e+07,4.666000e+09,NaN,-1.926000e+09,0.000000e+00,3.401690e+08,DASH
3419,2021-06-30,USD,6.182000e+09,5.179000e+09,3.334000e+09,4.668000e+09,NaN,2.450000e+08,7.250000e+08,3.060000e+08,...,NaN,3.210000e+08,9.590000e+08,1.400000e+07,4.619000e+09,NaN,-1.825000e+09,0.000000e+00,3.347070e+08,DASH
3420,2021-03-31,USD,5.902000e+09,5.025000e+09,4.007000e+09,4.474000e+09,NaN,2.820000e+08,6.540000e+08,2.500000e+08,...,NaN,2.590000e+08,8.450000e+08,1.300000e+07,4.555000e+09,NaN,-1.723000e+09,0.000000e+00,3.278150e+08,DASH


In [7]:
# Have a look at the cashflow statement data
cashflow_statements

,fiscalDateEnding,reportedCurrency,operatingCashflow,paymentsForOperatingActivities,proceedsFromOperatingActivities,changeInOperatingLiabilities,changeInOperatingAssets,depreciationDepletionAndAmortization,capitalExpenditures,changeInReceivables,...,dividendPayoutPreferredStock,proceedsFromIssuanceOfCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromIssuanceOfPreferredStock,proceedsFromRepurchaseOfEquity,proceedsFromSaleOfTreasuryStock,changeInCashAndCashEquivalents,changeInExchangeRate,netIncome,Symbol
0,2023-12-31,USD,3.989500e+10,NaN,NaN,-8.407000e+09,-9.530000e+09,2.848000e+09,2.392000e+09,-1.112400e+10,...,NaN,NaN,0.0,NaN,-2.013900e+10,NaN,NaN,NaN,3.391600e+10,AAPL
1,2023-09-30,USD,2.159800e+10,3.113000e+09,NaN,1.494600e+10,2.100600e+10,2.653000e+09,2.163000e+09,2.113700e+10,...,NaN,0.0,0.0,NaN,-2.100300e+10,NaN,8.390000e+08,NaN,2.295600e+10,AAPL
2,2023-06-30,USD,2.638000e+10,NaN,NaN,NaN,NaN,3.052000e+09,2.093000e+09,-3.661000e+09,...,NaN,0.0,0.0,NaN,0.000000e+00,NaN,2.769000e+09,NaN,1.988100e+10,AAPL
3,2023-03-31,USD,2.856000e+10,NaN,NaN,NaN,NaN,2.898000e+09,2.916000e+09,1.778600e+10,...,NaN,0.0,0.0,NaN,0.000000e+00,NaN,5.155000e+09,NaN,2.416000e+10,AAPL
4,2022-12-31,USD,3.400500e+10,7.030000e+08,NaN,-2.186000e+09,-6.890000e+08,2.916000e+09,3.787000e+09,-6.595000e+09,...,NaN,0.0,0.0,NaN,-1.947500e+10,NaN,-3.003000e+09,NaN,2.999800e+10,AAPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3428,2022-03-31,USD,-2.000000e+07,1.400000e+07,NaN,-1.000000e+07,-2.000000e+06,5.900000e+07,7.100000e+07,-2.500000e+07,...,NaN,NaN,NaN,NaN,5.000000e+06,NaN,-2.610000e+08,NaN,-1.670000e+08,DASH
3429,2021-12-31,USD,1.670000e+08,1.500000e+07,NaN,5.520000e+08,8.700000e+07,4.900000e+07,7.000000e+07,6.800000e+07,...,NaN,NaN,NaN,NaN,4.000000e+06,NaN,-3.560000e+08,NaN,-1.550000e+08,DASH
3430,2021-09-30,USD,1.070000e+08,1.100000e+07,NaN,8.000000e+07,5.500000e+07,4.100000e+07,5.900000e+07,4.000000e+07,...,NaN,NaN,NaN,NaN,5.000000e+06,NaN,-6.000000e+07,NaN,-1.010000e+08,DASH
3431,2021-06-30,USD,2.520000e+08,1.400000e+07,NaN,1.440000e+08,-8.000000e+06,3.700000e+07,5.400000e+07,-2.100000e+07,...,NaN,NaN,NaN,NaN,1.000000e+07,NaN,-6.500000e+07,NaN,-1.020000e+08,DASH


In [8]:
# Have a look at the FED fund rate data
fedfunds

,DATE,FEDFUNDS
0,1954-07-01,0.80
1,1954-08-01,1.22
2,1954-09-01,1.07
3,1954-10-01,0.85
4,1954-11-01,0.83
...,...,...
832,2023-11-01,5.33
833,2023-12-01,5.33
834,2024-01-01,5.33
835,2024-02-01,5.33


## 3. Feature Engineering <a class='anchor' id='chapter3'></a>

In this section, we calculate various financial key performance metrics (from the balance sheets, income statements and cashflow statements), technical indicators (from historical prices), and historical returns.

In [10]:
# Function to convert columns to a numeric data type
def convert_to_numeric(df, column_list):
    for column in column_list:
        df[column] = pd.to_numeric(df[column], errors='coerce')
    return df

# Function to calculate financial metrics out of the income statements, balance sheets and cashflow statements
def calculate_financial_metrics(income_statement, balance_sheet, cashflow_statement):
    # List of columns to convert to numeric in each DataFrame
    columns_to_convert_income = [
        'grossProfit', 'totalRevenue', 'operatingIncome', 'netIncome'
    ]
    columns_to_convert_balance = [
        'totalAssets', 'totalLiabilities', 'totalCurrentAssets', 'totalCurrentLiabilities', 'totalShareholderEquity', 'commonStockSharesOutstanding'
    ]
    columns_to_convert_cashflow = [
        'operatingCashflow', 'capitalExpenditures'
    ]

    # Convert columns to numeric
    income_statement = convert_to_numeric(income_statement, columns_to_convert_income)
    balance_sheet = convert_to_numeric(balance_sheet, columns_to_convert_balance)
    cashflow_statement = convert_to_numeric(cashflow_statement, columns_to_convert_cashflow)

    # Extracting from income_statement
    income_statement['grossProfitMargin'] = income_statement['grossProfit'] / income_statement['totalRevenue']
    income_statement['operatingMargin'] = income_statement['operatingIncome'] / income_statement['totalRevenue']
    income_statement['netProfitMargin'] = income_statement['netIncome'] / income_statement['totalRevenue']
    
    # Extracting from balance_sheet
    balance_sheet['currentRatio'] = balance_sheet['totalCurrentAssets'] / balance_sheet['totalCurrentLiabilities']
    balance_sheet['debtToEquityRatio'] = balance_sheet['totalLiabilities'] / balance_sheet['totalShareholderEquity']
    
    # Calculate earnings per share (EPS)
    income_statement['EPS'] = income_statement['netIncome'] / balance_sheet['commonStockSharesOutstanding']
    
    # Extracting from cashflow_statement
    cashflow_statement['operatingCashFlow'] = cashflow_statement['operatingCashflow']
    cashflow_statement['capitalExpenditures'] = cashflow_statement['capitalExpenditures']
    cashflow_statement['freeCashFlow'] = cashflow_statement['operatingCashflow'] - cashflow_statement['capitalExpenditures']
    
    # Merging the DataFrames to have a consolidated DataFrame
    # Assuming that the fiscalDateEnding columns are aligned and that these statements correspond to the same fiscal periods
    consolidated_df = pd.merge(income_statement, balance_sheet, on=('fiscalDateEnding','Symbol'))
    consolidated_df = pd.merge(consolidated_df, cashflow_statement, on=('fiscalDateEnding','Symbol'))
    
    # Selecting only the relevant columns for the final output
    relevant_columns = [
        'Symbol', 'fiscalDateEnding', 'grossProfitMargin', 'operatingMargin', 'netProfitMargin',
        'currentRatio', 'debtToEquityRatio', 'operatingCashFlow', 'EPS', 'capitalExpenditures', 'freeCashFlow', 'ebit'
    ]
    result_df = consolidated_df[relevant_columns]
    
    return result_df

In [11]:
# Function to merge the financial metrics with the historical price data
def merge_hp_fm(hist_prices, financial_metrics):
    # Ensure the date columns are in datetime format
    hist_prices['Date'] = pd.to_datetime(hist_prices['Date'])
    financial_metrics['fiscalDateEnding'] = pd.to_datetime(financial_metrics['fiscalDateEnding'])
    fedfunds['DATE'] = pd.to_datetime(fedfunds['DATE'])
    
    # Sort both DataFrames by 'Symbol' and then by their respective date columns
    hist_prices.sort_values(by=['Symbol', 'Date'], inplace=True)
    financial_metrics.sort_values(by=['Symbol', 'fiscalDateEnding'], inplace=True)

    # Perform a grouped merge
    merged_dfs = []  # To collect the merged DataFrames of each group
    for symbol in hist_prices['Symbol'].unique():
        # Filter rows for the current symbol in both DataFrames
        hp_sub = hist_prices[hist_prices['Symbol'] == symbol]
        fm_sub = financial_metrics[financial_metrics['Symbol'] == symbol]
        
        # Perform the asof merge for this symbol using 'Date' and 'fiscalDateEnding'
        merged_df_sub_1 = pd.merge_asof(hp_sub, fm_sub, left_on='Date', right_on='fiscalDateEnding', by='Symbol', direction='backward')
        merged_df_sub_2 = pd.merge_asof(merged_df_sub_1, fedfunds, left_on='Date', right_on='DATE', direction='backward')
        merged_dfs.append(merged_df_sub_2)
    
    # Concatenate all merged sub-DataFrames
    merged_df = pd.concat(merged_dfs)
    
    # Reset index to a range and clean up the DataFrame
    merged_df.reset_index(drop=True, inplace=True)
    merged_df.drop('fiscalDateEnding', axis=1, inplace=True)
    merged_df.drop('DATE', axis=1, inplace=True)

    return merged_df

In [45]:
# Function to convert prices to returns
def prices_to_returns(hist_prices):
    # Add the returns
    hist_prices['return_1d'] = hist_prices["Adj Close"].pct_change()
    hist_prices['return_2d'] = hist_prices["Adj Close"].pct_change(periods=2)
    hist_prices['return_3d'] = hist_prices["Adj Close"].pct_change(periods=3)
    hist_prices['return_4d'] = hist_prices["Adj Close"].pct_change(periods=4)
    hist_prices['return_5d'] = hist_prices['Adj Close'].pct_change(periods=5)
    hist_prices['return_6d'] = hist_prices['Adj Close'].pct_change(periods=6)
    hist_prices['return_7d'] = hist_prices['Adj Close'].pct_change(periods=7)
    hist_prices['return_8d'] = hist_prices['Adj Close'].pct_change(periods=8)
    hist_prices['return_9d'] = hist_prices['Adj Close'].pct_change(periods=9)
    hist_prices['return_2w'] = hist_prices['Adj Close'].pct_change(periods=10)
    hist_prices['return_3w'] = hist_prices['Adj Close'].pct_change(periods=15)
    hist_prices['return_1m'] = hist_prices['Adj Close'].pct_change(periods=21)
    hist_prices['return_2m'] = hist_prices['Adj Close'].pct_change(periods=42)
    hist_prices['return_3m'] = hist_prices['Adj Close'].pct_change(periods=63)
    hist_prices['return_6m'] = hist_prices['Adj Close'].pct_change(periods=126)
    hist_prices['return_1y'] = hist_prices['Adj Close'].pct_change(periods=252)
    hist_prices['return_2y'] = hist_prices['Adj Close'].pct_change(periods=504)

    # Calculate the relative difference between High and Low
    hist_prices['relative_diff'] = (hist_prices['High'] - hist_prices['Low']) / hist_prices['Low']
    hist_prices = hist_prices.drop('Open', axis = 'columns')
    hist_prices = hist_prices.drop('Close', axis = 'columns')

    return hist_prices

In [40]:
import ta

# Function to calculate technical indicators with the ta package
def calculate_technical_metrics(df):
    # Copy of the input df
    ta_df = df.copy()

    # Calculate the P/E Ratio
    ta_df['P/E Ratio'] = ta_df['Adj Close'] / ta_df['EPS']

    # Calculate the Simple Moving Average (SMA)
    ta_df['SMA_50'] = ta.trend.sma_indicator(ta_df['Adj Close'], window=50)

    # Calculate the Exponential Moving Average (EMA)
    ta_df['EMA_50'] = ta.trend.ema_indicator(ta_df['Adj Close'], window=50)

    # Calculate the Relative Strength Index (RSI)
    ta_df['RSI'] = ta.momentum.rsi(ta_df['Adj Close'], window=14)

    # Calculate the Bollinger Bands
    bollinger = ta.volatility.BollingerBands(close=ta_df['Adj Close'], window=20, window_dev=2)
    ta_df['Bollinger_High'] = bollinger.bollinger_hband()
    ta_df['Bollinger_Low'] = bollinger.bollinger_lband()

    # Moving Average Convergence Divergence MACD
    ta_df['MACD'] = ta.trend.macd_diff(ta_df['Adj Close'])

    # Stochastic Oscillator
    stoch = ta.momentum.StochasticOscillator(high=ta_df['High'], low=ta_df['Low'], close=ta_df['Adj Close'], window=14, smooth_window=3)
    ta_df['Stochastic_%K'] = stoch.stoch()
    ta_df['Stochastic_%D'] = stoch.stoch_signal()

    # Average True Range (ATR)
    if len(ta_df) >= 14:
        ta_df['ATR'] = ta.volatility.average_true_range(high=ta_df['High'], low=ta_df['Low'], close=ta_df['Adj Close'], window=14)

    # Ichimoku Cloud
    ichimoku = ta.trend.IchimokuIndicator(high=ta_df['High'], low=ta_df['Low'], window1=9, window2=26, window3=52)
    ta_df['Ichimoku_A'] = ichimoku.ichimoku_a()
    ta_df['Ichimoku_B'] = ichimoku.ichimoku_b()

    # Commodity Channel Index (CCI)
    ta_df['CCI'] = ta.trend.cci(high=ta_df['High'], low=ta_df['Low'], close=ta_df['Adj Close'], window=20)

    return ta_df

In [48]:
# Split the data frame by 'Symbol' and processing each group
def calculate_returns_ta(df):
    grouped = df.groupby('Symbol')
    results = []

    for name, group in grouped:
        group = group.sort_values('Date')  # Ensure the data is sorted by date
        
        # Apply the prices_to_returns function
        group_returns = prices_to_returns(group)
        
        # Apply the calculate_technical_metrics function
        group_ta = calculate_technical_metrics(group_returns)

        # Add 1 week, 2 week, 3 week and 1 month future returns
        group_ta['return_next_1w'] = group_ta['Adj Close'].pct_change(periods=5).shift(-5)
        group_ta['return_next_2w'] = group_ta['Adj Close'].pct_change(periods=5).shift(-10)
        group_ta['return_next_3w'] = group_ta['Adj Close'].pct_change(periods=5).shift(-15)
        group_ta['return_next_1m'] = group_ta['Adj Close'].pct_change(periods=21).shift(-21)

        results.append(group_ta)

    # Recombine the DataFrames
    final_df = pd.concat(results)
    return final_df

In [50]:
# Apply the the functions to calculate financial features
def feature_engineering(hist_prices, income_statements, balance_sheets, cashflow_statements, fedfunds=fedfunds):

    # Calculate financial metrics
    financial_metrics = calculate_financial_metrics(income_statements, balance_sheets, cashflow_statements)

    # Merge the prices and the financial metrics
    merged = merge_hp_fm(hist_prices, financial_metrics)

    # Add months (seasonal effects)
    merged['Date'] = pd.to_datetime(merged['Date'])
    merged['month'] = merged['Date'].dt.month
    
    # Calculate returns and technical indicators
    final_df = calculate_returns_ta(merged)

    # Convert the headers to lowercase
    final_df.columns = [column.lower() for column in final_df.columns]

    # Convert the 'symbol' column to lowercase
    final_df['symbol'] = final_df['symbol'].str.lower()

    # Drop some columns that are not needed for later analysis
    final_df.drop('relative_diff', axis=1, inplace=True)
    final_df.drop('volume', axis=1, inplace=True)
    final_df.drop('high', axis=1, inplace=True)
    final_df.drop('low', axis=1, inplace=True)
    final_df.drop('adj close', axis=1, inplace=True)

    return final_df
        
financial_data = feature_engineering(hist_prices, income_statements, balance_sheets, cashflow_statements)

# Drop na values
financial_data = financial_data.dropna()

# Have a look at the final data
financial_data

,date,symbol,grossprofitmargin,operatingmargin,netprofitmargin,currentratio,debttoequityratio,operatingcashflow,eps,capitalexpenditures,freecashflow,ebit,fedfunds,month,return_1d,return_2d,return_3d,return_4d,return_5d,return_6d,return_7d,return_8d,return_9d,return_2w,return_3w,return_1m,return_2m,return_3m,return_6m,return_1y,return_2y,p/e ratio,sma_50,ema_50,rsi,bollinger_high,bollinger_low,macd,stochastic_%k,stochastic_%d,atr,ichimoku_a,ichimoku_b,cci,return_next_1w,return_next_2w,return_next_3w,return_next_1m
504,2012-01-03,aapl,0.446831,0.374247,0.281959,1.582657,0.539976,1.755400e+10,14.013950,1.321000e+09,1.623300e+10,1.734000e+10,0.08,1,0.015383,0.015082,0.021334,0.011561,0.019587,0.031815,0.037281,0.038591,0.075927,0.079287,0.044739,0.060062,0.037124,0.097784,0.176860,0.247777,0.921545,0.887246,11.825374,11.845322,66.163985,12.476791,11.364478,0.067472,-84.815948,-102.041335,2.297438,14.067679,13.853929,156.486906,0.029205,0.013869,0.040899,0.106729
505,2012-01-04,aapl,0.446831,0.374247,0.281959,1.582657,0.539976,1.755400e+10,14.013950,1.321000e+09,1.623300e+10,1.734000e+10,0.08,1,0.005374,0.020839,0.020537,0.026822,0.016997,0.025066,0.037360,0.042855,0.044172,0.081708,0.055124,0.060918,0.040336,0.109906,0.175347,0.247969,0.928538,0.892014,11.837813,11.871021,67.540985,12.561841,11.341199,0.070048,-74.762182,-90.131285,2.303062,14.210447,13.892857,142.531088,0.022035,0.012306,0.039462,0.111842
506,2012-01-05,aapl,0.446831,0.374247,0.281959,1.582657,0.539976,1.755400e+10,14.013950,1.321000e+09,1.623300e+10,1.734000e+10,0.08,1,0.011102,0.016536,0.032173,0.031867,0.038223,0.028289,0.036447,0.048877,0.054433,0.055765,0.075153,0.063663,0.037115,0.105169,0.170297,0.251587,0.981467,0.901918,11.845227,11.901154,70.249254,12.668481,11.316437,0.075799,-60.651062,-73.409731,2.313385,14.316071,13.961964,146.542832,0.008038,-0.002587,0.064192,0.109896
507,2012-01-06,aapl,0.446831,0.374247,0.281959,1.582657,0.539976,1.755400e+10,14.013950,1.321000e+09,1.623300e+10,1.734000e+10,0.08,1,0.010454,0.021672,0.027162,0.042963,0.042654,0.049076,0.039038,0.047281,0.059842,0.065456,0.111023,0.080445,0.055367,0.119326,0.174279,0.265694,1.005888,0.911346,11.860121,11.935288,72.593663,12.789716,11.295918,0.082641,-50.871771,-62.095005,2.323770,14.488125,14.036964,156.594838,-0.006132,0.018104,0.059896,0.109920
508,2012-01-09,aapl,0.446831,0.374247,0.281959,1.582657,0.539976,1.755400e+10,14.013950,1.321000e+09,1.623300e+10,1.734000e+10,0.08,1,-0.001586,0.008851,0.020052,0.025533,0.041309,0.041000,0.047412,0.037390,0.045620,0.058161,0.112920,0.083888,0.055037,0.140427,0.191328,0.254701,0.989480,0.909900,11.872898,11.967288,71.661291,12.889010,11.290566,0.079895,-49.595079,-53.705971,2.336732,14.586518,14.126250,146.893501,0.007042,-0.010101,0.085797,0.130296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224745,2024-02-22,zg,0.757384,-0.221519,-0.154008,3.243048,0.469730,8.600000e+07,-0.880473,4.000000e+07,4.600000e+07,-1.050000e+08,5.33,2,-0.007552,-0.007360,-0.028436,-0.082035,-0.085311,-0.015559,-0.045090,-0.037739,-0.059115,-0.077408,-0.069704,-0.040082,-0.076244,0.370321,0.054093,0.199953,-0.176442,-58.207363,53.977000,52.151400,42.659430,57.715238,50.913762,-0.531221,4.773287,7.691142,2.143288,54.655000,51.160000,-128.723125,0.054829,0.000000,-0.017018,-0.028878
224746,2024-02-23,zg,0.757384,-0.221519,-0.154008,3.243048,0.469730,8.600000e+07,-0.880473,4.000000e+07,4.600000e+07,-1.050000e+08,5.33,2,0.003902,-0.003679,-0.003486,-0.024645,-0.078452,-0.081742,-0.011717,-0.041364,-0.033984,-0.055443,-0.091471,-0.027410,-0.112472,0.389039,0.054519,0.232631,-0.153783,-58.434515,54.129600,52.123894,43.423153,57.821678,50.600322,-0.523215,7.159931,7.120145,2.050910,54.914999,51.320000,-122.920169,0.080272,0.010255,-0.176848,-0.055588
224747,2024-02-26,zg,0.757384,-0.221519,-0.154008,3.243048,0.469730,8.

## 4. Export the financial features <a class='anchor' id='chapter4'></a>

In [51]:
# Save the data to a csv file to access it from other jupyter notebooks
financial_data.to_csv('./data/financial_data.csv', index=False)